추정과 가설검정
교차분석
t-test
분산분석
상관분석
요인분석
비모수통계분석

----

대응분석
포지셔닝분석
컨조인트분석
다차원척도법


reference
- [Handling ties in Wilcoxon Signed test](https://www.jstor.org/stable/2284536?seq=1#metadata_info_tab_contents)


1. 추정과 가설검정
2. t-test
3. 상관분석
4. 분산분석
5. 교차분석


6. 비모수통계분석 (적합도검정, 동질성검정, 상관성검정)

In [1]:
### scientific notation을 없애는 법
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
### 기본데이터 = 타이타닉
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00000,1,0,7.25000,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00000,1,0,71.28330,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00000,0,0,7.92500,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00000,1,0,53.10000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00000,0,0,8.05000,S,Third,man,True,NaN,Southampton,no,True


## 추정과 가설검정
- 가설: 연구가설을 설정한 뒤, 가설 검증을 위한 귀무가설과 대립가설을 설정한다. 
    - 연구가설: A는 B와 차이가 있을 것이다. (연구자가 확인하고 싶은 내용)
    - 귀무가설: A와 B는 같다.
    - 대립가설: A와 B는 같지 않다.

## t-Test (t 검정)
### 1. 일표본 t검정
- 단일 모집단에서 관심 그룹의 평균이 기준값과 같은지 비교할 때 사용
- 일표본 t검정은 모집단이 정규분포를 따른다고 가정 (*관심그룹의 정규성 검정이 선행되어야 함* but 일반적으로 표본이 20개 이상이면 정규성을 만족한다고 가정)

Step 1. 정규성 검정을 위한 Shapiro-Wilk test

In [3]:
from scipy.stats import shapiro

statistic, p_value = shapiro(df.fare)  ## titanic fare값의 평균이 30인지 확인해보자
print(statistic, p_value)

0.5218914747238159 1.0789998175301091e-43


step2-1. 모집단이 정규성을 만족하지 않을 때 - __윌콕슨 부호순위검정(Wilcoxon Signed Rank Test)__ 을 이용
- 가설
    - H0: 모집단의 중위수가 30(비교하고 싶은 값)이다
    - H1: 모집단의 중위수가 30(비교하고 싶은 값)이 아니다
- 계산방법  
    - |Xi - 30|(관측치에서 예상 중위수를 뺀 값)을 오름차순으로 줄 세워 순위를 매긴다, 절대값 안의 숫자가 음수가 되는 순위는 음수로 처리한다. 이 순위의 값을 전부 더했을 때, 모집단의 중위수가 추정치와 같았다면 순위의 합 W는 0에 가까워진다. 만약 W가 너무 크거나 작으면 귀무가설을 기각하게 된다.   
    - W는 평균은 0 분산은 n(n+1)(2n+1)/6의 통계량을 가지게 되는데, W는 특정한 분포를 갖는 값의 합이라는 점(W의 첫번째 값은 각 0.5의 확률로 1 또는 -1, 두번째 값은 각 0.5의 확률로 +-2가 된다)에서 중심극한정리를 이용해 정규분포에 근사시킬 수 있다. 따라서 W를 이용한 검정은 Z 통계량을 이용하여 기각 또는 채택하게 된다.
- 하이퍼파라미터
    - alternative 
        - 'two-sided'(default)
        - 'less' (H0: 실제 중위수 < 추정값)
        - 'greater' (H0: 중위수 > 추정값)
    - zero_method 
        - 'wilcoxon'(default): |X-mu|가 0이 되는 값을 순위산정 이전에 제거하고 줄어든 샘플로 검정
        - 'pratt': |X-mu|가 0이 되는 값은 순위산정에 포함한 뒤에 합을 구할 때 제거 (W는 중심극한정리에 의한 정규분포를 가정할 수 없게 됨)
        - 'zsplit': 0이 되는 값의 부호를 양수와 음수 중 하나로 결정해줌
        - 'pratt'은 다른 방법에 비해 더 보수적인 검정을 하게 됨
    - nan_policy
        - 'propagate'(default) : NaN이 있으면 결과값도 NaN으로 출력
        - 'omit': Nan값을 생략하고 나머지 값으로 계산한 결과값 출력
        - 'raise': NaN값이 있으면 ValueError 출력

In [7]:
###윌콕슨 부호순위 검정
from scipy.stats import wilcoxon

result = wilcoxon(df.fare - 30)

print(result.pvalue)
print(result.statistic)

1.369432543821431e-15
135268.0


Step2-2. 모집단이 정규성을 만족할 때 - __일표본 t검정__ 을 이용
- H0: 모집단의 평균 mu = 추정값
- 하이퍼파라미터
    - ttest_1samp(a, popmean, axis, nan_policy, alternative)
    - popmean은 H0의 추정평균, omit, alternative는 Wilcoxon과 동일

In [6]:
from scipy.stats import ttest_1samp
result = ttest_1samp(df.fare, 30)
print(result.pvalue)
print(result.statistic)

0.18583845591428397
1.3240136368613238


### 2. 대응표본 t 검정
- 단일 모집단에 대해 특정 이벤트 발생 이전과 이후의 차이를 비교
- 가설
    - 연구가설: 이벤트에 따른 값의 차이가 없을 것이다.
    - H0: 이전 집단과 이후 집단 값 차이의 평균은 0이다.
    - H1: 이전 집단과 이후 집단 값 차이의 평균은 0이 아니다. 
- t-statistic = ${차이값의 평균 \over 차이 평균의 표준오차} $
- (이벤트 발생 이전값 - 이벤트 발생 이후값)이 정규성을 만족하며 연속형이어야 한다는 기본가정이 존재. Shapiro Wilk test 또는 샘플의 개수를 기반으로 판단할 수 있음.
- 하이퍼파라미터
    - ttest_rel(a, b, axis, nan_policy, alternative)
    - nan_policy: ttest_1samp와 동일
    - alternative: ttest_1samp와 동일

In [8]:
test_data = {
    'before': [7,3,4,5,2,1,6,6,5,4],
    'after':[8,4,5,6,2,3,6,8,6,5]
}
test_data = pd.DataFrame(test_data)


from scipy.stats import ttest_rel

result = ttest_rel(test_data['before'], test_data['after'])
print(result.statistic)
print(result.pvalue)

-4.743416490252569
0.001053871257016553


### 3. 독립표본 t검정
- 두 개의 독립된 모집단의 평균을 비교 (ex. 성별 - 범주형데이터 에 따른 출근 준비시간 - 연속형 데이터 이 다른가?)
- 가설
    - 연구가설: 범주형 변수에 따른 연속형 변수의 차이가 존재할 것이다. 
    - H0: $\mu_{A} - \mu_{B} = 0$
- 두 모집단은 모두 정규성을 만족하고, 서로 독립이어야 함 (독립이다 = 두 집단이 서로 영향을 주고 있지 않다)
- 두 모집단의 등분산성 성립에 따라 검정통계량 계산 방법이 달라짐

Step1. 등분산성 여부를 확인 - __Levene's test__
- Levene's test의 가설
    - H0: $\sigma^{2}_{1} = \sigma^{2}_{2} = \sigma^{2}_{3} = ... = \sigma^{2}_{k}$ (모든 집단의 분산이 같다)
    - H1: $\sigma^{2}_{i} \neq \sigma^{2}_{j}$ for at least one pair (i,j) (적어도 한 집단의 분산이 같지 않다)  
- W= $(N−k)\over(k−1)$ $\Sigma_{i=1}^{k}N_{i} {(\bar{Z}_{i} - \bar{Z}_{..})^{2} } \over \Sigma_{i=1}^{k} \Sigma_{j=1}^{N_{i}}{(Z_{ij}−\bar{Z}_{i.})}^{2}$    (전체 표본크기 N이 범주형변수에 따른 그룹 k개로 쪼개졌고 $N_{i}$는 i번째 그룹의 표본을 의미한다. $Z_{ij}$를 구하는 방법에 따라 하이퍼파라미터가 달라진다.
- 하이퍼파라미터
    - levene(sample1, sample2, .... , center, proportiontocut)
    - center ($Z_{ij}$를 구하는 방법)
        - 'median'(default) 
            - $Z_{ij} = |Y_{ij} - \tilde{Y}_{i.}|$ 관측된 Y값에서 i번째 그룹의 중앙값을 뺀 값
            - 왜도가 높은(정규성을 보이지 않는) 데이터에 적합
        - 'mean'
            - $Z_{ij} = |Y_{ij} - \bar{Y}_{i.}|$  관측된 Y값에서 i번째 그룹의 평균을 뺀 값
            - 대칭이며 꼬리분포가 적당한 데이터에 적합 (정규성)
        - 'trimmed'
            - $Z_{ij} = |Y_{ij} - \bar{Y}_{i.}^{'}|$  관측된 Y 값에서 양쪽 10% 값을 제한 (trimmed data) 데이터들의 평균을 뺌
            - 두터운 꼬리 분포를 가지는 데이터에 적합
    - proportiontocut (default = 0.05)  
    : center이 'trimmed'일 때 양쪽에서 몇 퍼센트의 비율만큼 데이터를 제외할 것인지 결정

In [10]:
from scipy.stats import levene

result = levene(df[df['adult_male']==1]['fare'], df[df['adult_male']==0]['fare']) ## 남자와 여자의 표 값을 비교하기 위해 등분산 검정
print(result.statistic)
print(result.pvalue)

16.59972698234938
5.028279844812678e-05


Step2. 등분산성 여부에 맞게 독립표본 t 검정을 실행
- 하이퍼파라미터
    - ttest_ind(a, b, axis, equal_var, nan_policy, permutations, random_state, alternative, trim)
    - equal_var (default : True)
        - 두 모집단이 등분산성을 만족할 때 True
        - 두 모집단이 등분산성을 만족하지 않을 때 False (Welch's t-test로 계산됨)
    - permutations (default : None or 0)
        - None 일시 t 분포를 가정한 p-value를 계산함
        - 숫자만큼 무작위의 순열검정을 수행하여 p-value를 계산
        - random_state는 permutation test 수행 시 seed를 지정해줌
    - alternative (default: 'two-sided')
        - 'less': 첫번째 샘플의 평균 < 두번째 샘플의 평균
        - 'greater': 첫번째 샘플의 평균 > 두번째 샘플의 평균
    - trim (default: 0)
        - 해당 비율만큼 각 샘플의 양쪽 값을 제거하고 검정을 수행(Yuen's t-test or Trimmed t-test)

In [13]:
from scipy.stats import ttest_ind

result = ttest_ind(df[df['adult_male']==1]['fare'], df[df['adult_male']==0]['fare'], equal_var = False) ## 등분산성을 만족하지 않음
print(result.statistic)
print(result.pvalue)

-5.267344838807816
1.897839302718044e-07


## 분산분석
3개 이상의 그룹 간 평균 차이를 알고 싶을 때 분산분석을 사용  
### 1. 단일변량분산분석 - (1) 일원분산분석(One-way ANOVA)
- 단일변량분산분석은 종속변수가 __1개일 때__ 집단 간 종속변수의 차이를 분석
- 그 중 일원분산분석은 집단을 나누는 __독립변수가 1개__ 일 때 사용
    - ex) 고객 등급 별 매장방문 횟수 차이 검정
- 가설
    - H0: $\mu_{1} = \mu_{2} = ... = \mu_{k}$ (각 그룹의 모집단 평균은 모두 같다)
    - H1: $\mu_{i} \neq \mu_{j}\; for \;at \;least\; one \;(i,j)$ (적어도 한 그룹의 모집단 평균이 다르다)
- One-way ANOVA의 기본 가정으로 각 그룹의 데이터는 독립성, 정규성, 등분산성을 만족한다 (독립성은 일반적으로 존재한다고 봄)
- 그룹 간 분산과 그룹 내 분산의 크기를 비교하여 가설을 검정하며, F-통계량을 사용

Step1. 정규성, 등분산성 유무 확인 (정규성은 표본 개수로도 확인 가능)

In [22]:
##class에 따른 fare차이
a = df[df['pclass']==1]['fare']
b = df[df['pclass']==2]['fare']
c = df[df['pclass']==3]['fare']

print("정규성:", shapiro(a), shapiro(b), shapiro(c))
print('\n')
print("등분산성: ", levene(a, b, c))

정규성: ShapiroResult(statistic=0.718401312828064, pvalue=7.556354866266318e-19) ShapiroResult(statistic=0.7785055637359619, pvalue=2.1134516002102284e-15) ShapiroResult(statistic=0.6183854937553406, pvalue=1.5995106796893175e-31)


등분산성:  LeveneResult(statistic=118.57020037111805, pvalue=2.288618662046418e-46)


Step2-1. 등분산성을 만족하는 경우 
- f_oneway(a,b,..., axis)

In [23]:
from scipy.stats import f_oneway
result = f_oneway(a, b, c)
print(result.statistic)
print(result.pvalue)

242.34415651744814
1.0313763209141171e-84


Step2-2. 등분산성을 만족하지 않는 경우 (Welch's ANOVA)
- pingouin 패키지 활용 (설치되어 있지 않을 시 pip install pingouin 명령어로 설치)
- welch_anova(data, dv - 종속변수명, between - 독립변수인 범주형변수명)
- 결과값
    - 'Source': Factor names
    - 'SS': Sums of squares
    - 'DF': Degrees of freedom
    - 'MS': Mean squares
    - 'F': F-values
    - 'p-unc': uncorrected p-values
    - 'np2': Partial eta-squared

In [24]:
import pingouin as pg
pg.welch_anova(dv = 'fare', between = 'pclass', data = df)

,Source,ddof1,ddof2,F,p-unc,np2
0,pclass,2,333.63337,102.00059,0.00000,0.35309


g:\python\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
g:\python\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.8, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


Step3. 사후검정 (등분산성 만족 시 Tukey's Test, 만족하지 않을 시 Games Howell's test)
- 일원배치분산분석 결과 p-value가 0.05 아래로 귀무가설을 기각한다면, 어떤 집단에서 평균 차이가 발생하는지 사후검정이 필요
- 사후검정 방법
    - Tukey test (샘플 크기가 다를 시 Tukey-Kramer 검정)
        - 모든 그룹의 pairwise한 조합 간 평균을 비교한다
        - 등분산성 만족 필요
    - Scheffe test
        - 그룹 간 pairwise한 비교를 넘어 더 많은 조합을 고려한다(ex. A 그룹과 B그룹의 모평균 합 = C와 D의 모평균 합)
        - 따라서 신뢰구간이 넓어지며, 검정력이 낮아진다.
        - 등분산성 만족 필요
    - Bonferroni test (Dunn's test의 일종)
        - 그룹 간 표본 크기에 영향을 받지 않으나, 특정 그룹 간의 비교를 사전에 생각해두었을 때 사용함
        - 등분산성 만족 필요
    - Games Howell's test
        - 등분산성 필요하지 않음
        - Welch's ANOVA 사후검정에 최적화되어 있으며 

In [25]:
## Tukey-Kramer 방식
from statsmodels.stats.multicomp import pairwise_tukeyhsd

posthoc = pairwise_tukeyhsd(df['fare'], df['pclass'], alpha = 0.05)
print(posthoc)

 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     1      2 -63.4925  0.001 -72.9167 -54.0683   True
     1      3 -70.4791  0.001 -78.1491 -62.8092   True
     2      3  -6.9866 0.1081 -15.1066   1.1333  False
------------------------------------------------------


In [30]:
## Scheffe 방식 (pip install scikit-posthocs)
## posthoc_scheffe(a = array or df, val_col = 종속변수, group_col = 독립변수, sort = True or False)
## return 값 : p-value
import scikit_posthocs as sp
sp.posthoc_scheffe(df, val_col = 'fare', group_col = 'pclass')

,3,1,2
3,1.00000,0.00000,0.13060
1,0.00000,1.00000,0.00000
2,0.13060,0.00000,1.00000


In [34]:
## Dunn's test
## posthoc_dunn(a = array or df, val_col = 종속변수, group_col = 독립변수, p_adjust = 'bonferroni' 외 p-value 계산 방법들 , sort = boolean)
## return 값: p-value
import scikit_posthocs as sp
sp.posthoc_dunn(df, val_col = 'fare', group_col = 'pclass', p_adjust = 'bonferroni')

,1,2,3
1,1.00000,0.00000,0.00000
2,0.00000,1.00000,0.00000
3,0.00000,0.00000,1.00000


In [33]:
### Games Howell's test
from pingouin import pairwise_gameshowell
pairwise_gameshowell(data = df, dv = 'fare', between = 'pclass').round(3)

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,1,2,84.15500,20.66200,63.49300,5.42400,11.70600,229.72700,0.00100,1.17200
1,1,3,84.15500,13.67600,70.47900,5.36000,13.15000,219.28300,0.00100,1.07300
2,2,3,20.66200,13.67600,6.98700,1.12300,6.22200,294.77000,0.00100,0.53700


### 단일변량분산분석 - (2) 이원분산분석(Two-way ANOVA)